1)Если еще не сделано, то: Модифицировать приложение из предыдущего домашнего задания:
- приложение должно собирать вакансии сразу с двух сайтов hh.ru и superjob.ru
- собранные данные должны быть приведены к общей структуре
- разделить зарплату на две составляющие (мин. и макс.) и сохранить в виде int. Если валюта указана другая, привести все к рублям.

2)Реализовать сохранение полученных вакансий в СУБД (на выбор SQLite или MongoDB)
3)Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы.
3*)Модифицировать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [78]:
import pandas as pd
from pymongo import MongoClient
from pprint import pprint

#### Загрузка датасетов

In [79]:
HH_PATH = 'hh программист с++.csv'
SUPERJOB_PATH = 'superjob программист c#.csv'

In [80]:
hh_df = pd.read_csv(HH_PATH, encoding='utf-8')
hh_df.head(3)

,vacansy,min_salary,max_salary,url,source,employer
0,Программист С++/Qt,70000.0,NaN,https://hh.ru/vacancy/31273030?query=%D0%BF%D1...,hh.ru,Cappasity Inc.
1,Программист С++ (графика),80000.0,NaN,https://hh.ru/vacancy/32543357?query=%D0%BF%D1...,hh.ru,Cappasity Inc.
2,Программист С++,NaN,100000.0,https://hh.ru/vacancy/32417827?query=%D0%BF%D1...,hh.ru,Группа Компаний «АМЕ»


In [81]:
sj_df = pd.read_csv(SUPERJOB_PATH, encoding='utf-8')
sj_df.head(3)

,vacansy,min_salary,max_salary,url,source,employer
0,Инженер-программист / Разработчик C#,300000.0,NaN,https://www.superjob.ru/vakansii/inzhener-prog...,superjob.ru,ФГУ НПП Гамма
1,Инженер-программист C#,NaN,NaN,https://www.superjob.ru/vakansii/inzhener-prog...,superjob.ru,Центральная ППК
2,.NET Developer / Разработчик C#,NaN,NaN,https://www.superjob.ru/vakansii/net-developer...,superjob.ru,Информационные технологии


In [82]:
hh_df.dtypes

vacansy        object
min_salary    float64
max_salary    float64
url            object
source         object
employer       object
dtype: object

In [83]:
sj_df.dtypes

vacansy        object
min_salary    float64
max_salary    float64
url            object
source         object
employer       object
dtype: object

#### Заполним NaN значения нулями

In [84]:
values = {'min_salary': 0, 'max_salary': 0}
hh_df = hh_df.fillna(value=values,);
sj_df = sj_df.fillna(value=values);

In [85]:
hh_df.head(3)

,vacansy,min_salary,max_salary,url,source,employer
0,Программист С++/Qt,70000.0,0.0,https://hh.ru/vacancy/31273030?query=%D0%BF%D1...,hh.ru,Cappasity Inc.
1,Программист С++ (графика),80000.0,0.0,https://hh.ru/vacancy/32543357?query=%D0%BF%D1...,hh.ru,Cappasity Inc.
2,Программист С++,0.0,100000.0,https://hh.ru/vacancy/32417827?query=%D0%BF%D1...,hh.ru,Группа Компаний «АМЕ»


In [86]:
sj_df.head(3)

,vacansy,min_salary,max_salary,url,source,employer
0,Инженер-программист / Разработчик C#,300000.0,0.0,https://www.superjob.ru/vakansii/inzhener-prog...,superjob.ru,ФГУ НПП Гамма
1,Инженер-программист C#,0.0,0.0,https://www.superjob.ru/vakansii/inzhener-prog...,superjob.ru,Центральная ППК
2,.NET Developer / Разработчик C#,0.0,0.0,https://www.superjob.ru/vakansii/net-developer...,superjob.ru,Информационные технологии


#### Подключаемся к БД и создаем записи

In [87]:
client = MongoClient('localhost',27017)
db = client['vacancys']

#### Вставляем данные в БД

In [92]:
data = hh_df.to_dict(orient='records')
hh_db = db.hh_cpp
hh_db.insert_many(data)

In [93]:
hh_db.count()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


165

In [94]:
data = sj_df.to_dict(orient='records')
sj_db = db.sj_csharp
sj_db.insert_many(data)

In [95]:
sj_db.count()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


200

#### Функция поиска и вывода на экран вакансий с суммой, вводимой пользователем

In [96]:
def showVacancys(col_name):
    salary = input('Введите сумму: ')
    for post in col_name.find({'max_salary': {'$gt':int(salary)}}):
        pprint(post)

#### Демонстрация работы функции

In [97]:
showVacancys(hh_db)

Введите сумму: 100000
{'_id': ObjectId('5d7fbdefa88bb059034219a6'),
 'employer': ' МСК БЛ ГРУПП',
 'max_salary': 150000.0,
 'min_salary': 0.0,
 'source': 'hh.ru',
 'url': 'https://hh.ru/vacancy/33514106?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%20%D1%81%2B%2B',
 'vacansy': 'Программист С++'}
{'_id': ObjectId('5d7fbdefa88bb059034219ac'),
 'employer': ' ФОРМ',
 'max_salary': 170000.0,
 'min_salary': 115000.0,
 'source': 'hh.ru',
 'url': 'https://hh.ru/vacancy/33289931?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%20%D1%81%2B%2B',
 'vacansy': 'Ведущий инженер-программист С++'}
{'_id': ObjectId('5d7fbdefa88bb059034219b0'),
 'employer': ' ЮРИОН',
 'max_salary': 110000.0,
 'min_salary': 90000.0,
 'source': 'hh.ru',
 'url': 'https://hh.ru/vacancy/33483197?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%20%D1%81%2B%2B',
 'vacansy': 'Программист С++'}
{'_id': ObjectId('5d7fbdefa88bb059034219b2'),
 'employer': 'ОО

In [98]:
showVacancys(sj_db)

Введите сумму: 100000
{'_id': ObjectId('5d7fbdf4a88bb05903421a4c'),
 'employer': 'Минерал-Инфо',
 'max_salary': 200000.0,
 'min_salary': 150000.0,
 'source': 'superjob.ru',
 'url': 'https://www.superjob.ru/vakansii/web-programmist-29723693.html',
 'vacansy': 'Web-программист / Web-разработчик'}
{'_id': ObjectId('5d7fbdf4a88bb05903421a50'),
 'employer': 'Кадровый Центр на Сокольнической Площади',
 'max_salary': 250000.0,
 'min_salary': 150000.0,
 'source': 'superjob.ru',
 'url': 'https://www.superjob.ru/vakansii/razrabotchik-c-32357613.html',
 'vacansy': 'Разработчик C#'}
{'_id': ObjectId('5d7fbdf4a88bb05903421a51'),
 'employer': '«Группа ГУТА»',
 'max_salary': 190000.0,
 'min_salary': 150000.0,
 'source': 'superjob.ru',
 'url': 'https://www.superjob.ru/vakansii/full-stack-developer-32491461.html',
 'vacansy': 'Full-Stack developer (Angular, C#, .NET)'}
{'_id': ObjectId('5d7fbdf4a88bb05903421a58'),
 'employer': 'НПП Гарант Сервис Университет',
 'max_salary': 140000.0,
 'min_salary': 120

In [99]:
hh_df.tail()

,vacansy,min_salary,max_salary,url,source,employer
160,Системный программист С (Linux),0.0,0.0,https://hh.ru/vacancy/22958995?query=%D0%BF%D1...,hh.ru,"Русские базовые информационные технологии, На..."
161,System Engineer Embedded Linux,0.0,0.0,https://hh.ru/vacancy/33059759?query=%D0%BF%D1...,hh.ru,InterJob (Анташкевич И. О.)
162,Системный программист в отдел адаптации Sailfi...,0.0,0.0,https://hh.ru/vacancy/29727812?query=%D0%BF%D1...,hh.ru,Открытая Мобильная Платформа
163,Старший математик-программист,0.0,0.0,https://hh.ru/vacancy/27444585?query=%D0%BF%D1...,hh.ru,НСК
164,Архитектор 1С УХ,80.0,0.0,https://hh.ru/vacancy/33077576?query=%D0%BF%D1...,hh.ru,ООО Вератис


In [69]:
hh_df2.drop(hh_df2.index, inplace=True)

In [100]:
hh_df2 = pd.DataFrame({'vacansy':['new_vacancy','new_vacancy2'], 
                       'min_salary':[999999,999999],
                       'max_salary':[999999,999999],
                       'url':['new_url','new_url2'],
                       'source':['hh.ru','hh.ru'],
                       'employer':['new_employer','new_employer2']}) 

In [101]:
hh_df2.head(10)

,vacansy,min_salary,max_salary,url,source,employer
0,new_vacancy,999999,999999,new_url,hh.ru,new_employer
1,new_vacancy2,999999,999999,new_url2,hh.ru,new_employer2


In [102]:
hh_df = hh_df.append(hh_df2, ignore_index = True) 

In [103]:
hh_df.tail()

,vacansy,min_salary,max_salary,url,source,employer
162,Системный программист в отдел адаптации Sailfi...,0.0,0.0,https://hh.ru/vacancy/29727812?query=%D0%BF%D1...,hh.ru,Открытая Мобильная Платформа
163,Старший математик-программист,0.0,0.0,https://hh.ru/vacancy/27444585?query=%D0%BF%D1...,hh.ru,НСК
164,Архитектор 1С УХ,80.0,0.0,https://hh.ru/vacancy/33077576?query=%D0%BF%D1...,hh.ru,ООО Вератис
165,new_vacancy,999999.0,999999.0,new_url,hh.ru,new_employer
166,new_vacancy2,999999.0,999999.0,new_url2,hh.ru,new_employer2


In [132]:
def col_update(col_name, df):
    data = df.to_dict(orient='records')
    for i,item in enumerate(data): 
        col_name.update_one({ 'url': { 'url': data[i]['url'] } ,
                                'vacansy': { 'vacansy': data[i]['vacansy'] } ,
                                'min_salary': { 'min_salary': data[i]['min_salary'] } ,
                                'max_salary': { 'max_salary': data[i]['max_salary'] } ,
                                'source': { 'source': data[i]['source'] } ,
                                'employer': { 'employer': data[i]['employer'] }},
                            {'$set': { "vacansy" : data[i]['vacansy'],
                                "min_salary" : data[i]['min_salary'],
                                "max_salary" : data[i]['max_salary'],
                                "url" : data[i]['url'],
                                "source" : data[i]['source'],
                                "employer" : data[i]['employer']
                                     }
                            } ,upsert=True)

In [133]:
col_update(hh_db, hh_df2)

In [135]:
showVacancys(hh_db)

Введите сумму: 999000
{'_id': ObjectId('5d7fe41599dae9ef1e8fcb9a'),
 'employer': 'new_employer',
 'max_salary': 999999,
 'min_salary': 999999,
 'source': 'hh.ru',
 'url': 'new_url',
 'vacansy': 'new_vacancy'}
{'_id': ObjectId('5d7fe41599dae9ef1e8fcb9c'),
 'employer': 'new_employer2',
 'max_salary': 999999,
 'min_salary': 999999,
 'source': 'hh.ru',
 'url': 'new_url2',
 'vacansy': 'new_vacancy2'}
